# 📘 FastAPI 완전 정복 노트

이 노트북은 여러분이 작성한 모든 코드를 직접 실행해보고 검증할 수 있는 통합 실습 환경입니다.
DB 생성부터 API 호출, 보안 검증, 그리고 마이그레이션까지 순서대로 진행하며 전체 흐름을 익혀봅시다!

## 1. 🛠 환경 설정 (Imports & Setup)

프로젝트의 모든 모듈을 불러올 수 있도록 경로를 설정하고 필요한 라이브러리를 임포트합니다.

In [ ]:
import sys
import os
from typing import AsyncGenerator

# 현재 프로젝트 루트를 파이썬 경로에 추가
current_dir = os.getcwd()
if current_dir not in sys.path:
    sys.path.append(current_dir)

import asyncio
import nest_asyncio
nest_asyncio.apply()

import pytest
from httpx import AsyncClient, ASGITransport
from sqlmodel import SQLModel, select
from sqlalchemy.ext.asyncio import create_async_engine, AsyncSession
from sqlalchemy.orm import sessionmaker

# FastAPI 앱 및 모델 임포트
from fastapi import FastAPI
from appserver.app import include_routers
from appserver.db import use_session
from appserver.apps.account.models import User
from appserver.apps.account.schemas import UserCreatePayload, LoginPayload
from appserver.apps.account.utils import hash_password, verify_password
from dataclasses import dataclass

print("✅ 모든 라이브러리 임포트 완료!")

## 2. 🗄 데이터베이스 구축 (In-Memory DB)

실제 서버를 띄우지 않고, 노트북 안에서만 사용할 "가짜 DB(메모리 DB)"를 만듭니다.
이 단계가 성공해야 데이터를 저장하고 불러올 수 있습니다.

In [ ]:
# 비동기 엔진 생성 (메모리 DB)
test_engine = create_async_engine("sqlite+aiosqlite:///:memory:", echo=False)

async def init_db():
    async with test_engine.begin() as conn:
        # 모든 테이블 생성 (User 테이블 등)
        await conn.run_sync(SQLModel.metadata.create_all)
    print("✅ 데이터베이스 테이블 생성 완료!")

# 주피터는 이미 비동기 루프가 돌고 있으므로 await로 바로 실행
await init_db()

## 3. 🛡 로직 단위 테스트: 비밀번호 해싱

`appserver/apps/account/utils.py`의 핵심 보안 기능을 먼저 검증합니다.
내가 입력한 비밀번호가 DB에 어떻게 저장되는지 눈으로 확인하세요.

In [ ]:
raw_pw = "ilovepython"
hashed_pw = hash_password(raw_pw)

print(f"🔑 원본 비밀번호: {raw_pw}")
print(f"🔒 해시된 비밀번호: {hashed_pw}")

# 검증 테스트
assert verify_password(raw_pw, hashed_pw) == True, "비밀번호 검증 실패!"
assert verify_password("wrongpw", hashed_pw) == False, "틀린 비밀번호가 통과됨!"
print("✅ 비밀번호 보안 로직 정상 작동!")

## 4. 🌐 가상 API 클라이언트 생성

브라우저 없이 코드로 서버와 통신할 수 있는 `AsyncClient`를 만듭니다.
이 클라이언트가 여러분 대신 `/account/signup`, `/account/login` 등으로 요청을 보낼 겁니다.

In [ ]:
# 1. FastAPI 앱 인스턴스 생성
app = FastAPI()
include_routers(app)

# 2. DB 의존성 오버라이드 (가짜 DB 사용하도록 연결)
async def override_use_session() -> AsyncGenerator[AsyncSession, None]:
    async_session = sessionmaker(
        test_engine, class_=AsyncSession, expire_on_commit=False
    )
    async with async_session() as session:
        yield session

app.dependency_overrides[use_session] = override_use_session

# 3. 클라이언트 생성
client = AsyncClient(transport=ASGITransport(app=app), base_url="http://test")
print("✅ 가상 API 클라이언트 준비 완료!")

## 5. 🚀 실전 시나리오 1: 회원가입 (Sign Up)

유저가 회원가입 폼을 작성해서 제출하는 상황을 시뮬레이션합니다.
직접 JSON 데이터를 만들어서 POST 요청을 보내봅니다.

In [ ]:
signup_data = {
    "username": "fastapi_master",
    "password": "code1234",
    "password_again": "code1234",
    "email": "master@example.com",
    "display_name": "파이썬고수"
}

response = await client.post("/account/signup", json=signup_data)

print(f"📡 응답 상태 코드: {response.status_code}")
print(f"📄 응답 데이터: {response.json()}")

if response.status_code == 201:
    print("✅ 회원가입 성공!")
else:
    print("❌ 회원가입 실패 (로그 확인 필요)")

## 6. 🔍 DB 확인: 진짜 저장되었나요?

API는 성공했다고 하지만, 진짜 DB에 데이터가 들어갔는지 의심스러우시죠?
직접 SELECT 쿼리를 날려서 확인해봅시다.

In [ ]:
async_session = sessionmaker(test_engine, class_=AsyncSession, expire_on_commit=False)

async with async_session() as session:
    # SELECT * FROM user WHERE username = 'fastapi_master'
    stmt = select(User).where(User.username == "fastapi_master")
    result = await session.execute(stmt)
    user = result.scalar_one_or_none()

    if user:
        print(f"✅ DB 조회 성공!")
        print(f"ID: {user.id}")
        print(f"Username: {user.username}")
        print(f"Email: {user.email}")
        print(f"Hashed Password: {user.hashed_password} (안전하게 암호화됨)")
    else:
        print("❌ DB에 유저가 없습니다!")

## 7. 🚀 실전 시나리오 2: 로그인 (Login)

이제 회원가입한 아이디로 로그인을 시도합니다.
성공하면 **Access Token(신분증)**을 쿠키나 바디로 받아야 합니다.

In [ ]:
login_data = {
    "username": "fastapi_master",
    "password": "code1234"
}

response = await client.post("/account/login", json=login_data)

print(f"📡 응답 상태 코드: {response.status_code}")
print(f"📄 응답 바디: {response.json()}")

# 쿠키 확인
cookies = response.cookies
auth_token = cookies.get("auth_token")

if auth_token:
    print(f"\n🍪 발급된 쿠키(auth_token): {auth_token[:20]}... (생략)")
    print("✅ 로그인 및 토큰 발급 성공!")
else:
    print("❌ 쿠키가 없습니다. 로그인 로직을 점검하세요.")

## 8. 🚀 실전 시나리오 3: 내 정보 수정 (Update)

로그인 상태(`client`가 쿠기를 기억함)에서 닉네임을 변경해봅니다.
Dependency Injection(`get_current_user`)이 토큰을 어떻게 해석해서 유저를 찾아내는지 체험합니다.

In [ ]:
# 1. 현재 정보 확인 (@me)
me_res = await client.get("/account/@me")
print(f"변경 전 닉네임: {me_res.json()['display_name']}")

# 2. 정보 수정 요청 (PATCH)
update_data = {"display_name": "AI마스터"}
patch_res = await client.patch("/account/@me", json=update_data)

# 3. 변경 후 정보 확인
me_res_after = await client.get("/account/@me")
print(f"변경 후 닉네임: {me_res_after.json()['display_name']}")

assert me_res_after.json()['display_name'] == "AI마스터"
print("✅ 내 정보 수정 성공!")

## 9. 🧹 테스트 리소스 정리
사용했던 자원을 정리합니다.

In [ ]:
await client.aclose()
await test_engine.dispose()
print("✅ 테스트 종료 및 리소스 해제 완료")

---

## 10. 🏛 데이터 모델링 & 마이그레이션 (Alembic)

지금까지는 코드가 꺼지면 데이터가 사라지는 '메모리 DB'를 사용했습니다.
이제는 **파일로 남는 진짜 DB(local.db)**를 사용하고, 테이블 구조를 변경하는 실습을 해보겠습니다.

### 실습 목표: `Booking` 테이블에 `memo` 필드 추가하기

In [ ]:
import os
from sqlmodel import Field, Text

model_path = "appserver/apps/calendar/models.py"

# 1. Booking 모델 코드에 새로운 필드('memo')를 삽입하는 파이썬 스크립트
#    (마치 우리가 에디터에서 코드를 수정한 것처럼 파일 내용을 바꿉니다)
with open(model_path, "r", encoding="utf-8") as f:
    content = f.read()

if "memo: str = Field" not in content:
    target_string = 'description: str = Field(sa_type=Text, description="예약 설명")'
    new_field = '\n    memo: str = Field(default="", description="메모")'
    
    new_content = content.replace(target_string, target_string + new_field)
    
    with open(model_path, "w", encoding="utf-8") as f:
        f.write(new_content)
    print("✅ models.py 파일에 'memo' 필드가 추가되었습니다!")
else:
    print("ℹ️ 이미 'memo' 필드로 보이는 코드가 존재합니다.")

## 11. 📦 마이그레이션 명령어 실행

코드가 바뀌었으니, 이를 DB에 알리는 `Alembic` 명령어를 실행합니다.
노트북에서 `!`를 앞에 붙이면 터미널 명령어를 실행할 수 있습니다.

In [ ]:
# 1. 기존 테이블 최신화 (혹시 안 된 것이 있다면)
!alembic upgrade head

# 2. 새로운 변경사항(리비전) 생성
#    --autogenerate: 코드를 보고 바뀐 점을 자동으로 찾아라
!alembic revision --autogenerate -m "Add memo field to booking"

# 3. DB에 반영 (테이블 컬럼 추가)
!alembic upgrade head

## 12. ✅ 체크포인트: DB 파일 확인

이제 프로젝트 폴더에 `local.db` 파일이 생겼거나 업데이트되었을 것입니다.
이로써 여러분은 메모리 DB, 가상 클라이언트, 그리고 실제 DB 마이그레이션까지 **백엔드 개발의 전 과정**을 경험했습니다.

수고하셨습니다! 🎉